In [1]:
from src.data.process_utils import parse_file, compile_csv_logs, combine_data_files, remove_tags, create_tag_range
import os
from pathlib import Path

project_directory = Path(os.getcwd()).parents[0]

## Step 1: Review detection counts in raw data files
In this first step we will extract the detections from each raw datafile. Counts of line detections are noted to a log file. Low or no detections in a file indicate that the file may not be formatted correctly. Comparing counts between files also allows us to determine file duplicates.

In [3]:
# Inputs
file_list = [
    'INL 2023-04-04 11-23-27.txt',
    'INL 2023-04-11.txt',
    'INL 2023-04-18 13-49-26.txt',
    'INL 2023-04-27 12-09-44.txt',
    'INL 2023-05-08 11-40-17.txt',
    'INL 2023-05-09 12-05-47.txt',
    'INL 2023-05-17 14-56-03.txt',
    'INL 2023-06-02 14-43-21.txt',
    'INL 2023-06-13 12-24-05.txt',
    'INL re-try 2023-04-11.txt',
    'MST 05-08 retry.txt',
    'MST 2023-04-04 10-02-34.txt',
    'MST 2023-04-11.txt',
    'MST 2023-04-18 10-24-34.txt',
    'MST 2023-04-27.txt',
    'MST 2023-05-08 10-21-50.txt',
    'MST 2023-05-17 14-12-07.txt',
    'MST 2023-06-02 13-58-22.txt',
    'MST 2023-06-13 11-15-03.txt',
    'Main St 1 2023-04-18 10-24-34.txt',
    'Main st 2023-04-27.txt',
    'Main st re-try 2023-04-11.txt',
    'PAL 2023-04-04 10-36-49.txt',
    'PAL 2023-04-11.txt',
    'PAL 2023-04-18 12-31-22.txt',
    'PAL 2023-04-27 11-54-04.txt',
    'PAL 2023-05-08 11-26-16.txt',
    'PAL 2023-05-17 14-40-35.txt',
    'PAL 2023-06-02 14-25-46.txt',
    'PAL 2023-06-13 09-39-00.txt',
    'PAL re-try 2023-04-11.txt',
    'PAL retry 4-27.txt',
    'PLS 2023-04-18 12-31-22.txt']

input_directory = project_directory / 'data/raw/2023'
interim_directory = project_directory / 'data/interim/2023'

In [3]:
# Parses each raw file. Removes the lines that do not look like detection information.
for file in file_list:
    parse_file(input_file=project_directory / input_directory / file, output_dir=interim_directory)

# Complies the log generated for each file into one log. Logs contain counts of detections.
compile_csv_logs(interim_directory, 'processing_logs_compiled.csv')

Processing of "INL 2023-04-04 11-23-27.txt" complete.
Processing of "INL 2023-04-11.txt" complete.
Processing of "INL 2023-04-18 13-49-26.txt" complete.
Processing of "INL 2023-04-27 12-09-44.txt" complete.
Processing of "INL 2023-05-08 11-40-17.txt" complete.
Processing of "INL 2023-05-09 12-05-47.txt" complete.
Processing of "INL 2023-05-17 14-56-03.txt" complete.
Processing of "INL 2023-06-02 14-43-21.txt" complete.
Processing of "INL 2023-06-13 12-24-05.txt" complete.
Processing of "INL re-try 2023-04-11.txt" complete.
Processing of "MST 05-08 retry.txt" complete.
Processing of "MST 2023-04-04 10-02-34.txt" complete.
Processing of "MST 2023-04-11.txt" complete.
Processing of "MST 2023-04-18 10-24-34.txt" complete.
Processing of "MST 2023-04-27.txt" complete.
Processing of "MST 2023-05-08 10-21-50.txt" complete.
Processing of "MST 2023-05-17 14-12-07.txt" complete.
Processing of "MST 2023-06-02 13-58-22.txt" complete.
Processing of "MST 2023-06-13 11-15-03.txt" complete.
Processing 

## Step 2: Combining the detection data and filtering for valid tag IDs
Once we have reviewed the logs and selected the files want to combine we will combine all the detection files and remove unrecognized tags.

In [4]:
# Combine the detection data into a dataframe
files_to_combine = [
    'INL 2023-04-04 11-23-27_detection.csv',
    'INL 2023-04-11_detection.csv',
    'INL 2023-04-18 13-49-26_detection.csv',
    'INL 2023-04-27 12-09-44_detection.csv',
    'INL 2023-05-08 11-40-17_detection.csv',
    'INL 2023-05-09 12-05-47_detection.csv',
    'INL 2023-05-17 14-56-03_detection.csv',
    'INL 2023-06-02 14-43-21_detection.csv',
    'INL 2023-06-13 12-24-05_detection.csv',
    'MST 05-08 retry_detection.csv',
    'MST 2023-04-04 10-02-34_detection.csv',
    'MST 2023-04-11_detection.csv',
    'MST 2023-04-18 10-24-34_detection.csv',
    'MST 2023-04-27_detection.csv',
    'MST 2023-05-17 14-12-07_detection.csv',
    'MST 2023-06-02 13-58-22_detection.csv',
    'MST 2023-06-13 11-15-03_detection.csv',
    'PAL 2023-04-04 10-36-49_detection.csv',
    'PAL 2023-04-11_detection.csv',
    'PAL 2023-04-18 12-31-22_detection.csv',
    'PAL 2023-05-08 11-26-16_detection.csv',
    'PAL 2023-05-17 14-40-35_detection.csv',
    'PAL 2023-06-02 14-25-46_detection.csv',
    'PAL 2023-06-13 09-39-00_detection.csv',
    'PAL retry 4-27_detection.csv',]

processed_directory = project_directory / 'data/processed/2023'

# Concat files into a dataframe
df_combined = combine_data_files(interim_directory, files_to_combine)

# Remove site marker tags
l_marker_tags = ['0000_0000000000005126', '0000_0000000000012627', '0000_0000000000012617']
df_mt_removed = remove_tags(df_combined, l_marker_tags)

# Create list of valid fish tags for 2023
valid_tags_2023_1 = create_tag_range('900_228000487', (900, 995), pad=3)
valid_tags_2023_2 = create_tag_range('900_228000498', (0,211), pad=3)
# Add tags for 2022 that may have returned
valid_tags_2022_1 = create_tag_range('900_228000487', (501, 899), pad=3)
valid_tags_2022_2 = create_tag_range('900_228000487', (996,999), pad=3)

valid_tags = valid_tags_2023_1 + valid_tags_2023_2 + valid_tags_2022_1 + valid_tags_2022_2

# Filter for valid 2023 fish tags
mask_valid = df_mt_removed['TAG'].isin(valid_tags)
df_valid_fish = df_mt_removed[mask_valid].reset_index(drop=True)

# Check for duplicates on datetime, tag_id, and site
m_dups = df_valid_fish.duplicated(subset=['ARR','TAG', 'SCD'], keep=False)
if (n_dups := m_dups.sum()) > 0:
    print(f"There are {n_dups} duplicate records.")
else:
    print("No duplicates detected.")

# Sort the data by datetime and site
import pandas as pd
df_valid_fish['ARR']= pd.to_datetime(df_valid_fish['ARR'])
df_valid_fish = df_valid_fish.sort_values(['SCD', 'ARR'])

There are 145 duplicate records.


# Step3: Inspecting and Removing Duplicates

In [5]:
# Inspecting Duplicates
df_valid_fish[m_dups]

/var/folders/0t/j7f7wnld0qd6n2_fcq18rhfm0000gq/T/ipykernel_19241/3873119546.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_valid_fish[m_dups]


,DTY,ARR,TRF,DUR,TTY,ANT,TAG,SCD,NCD,EFA,TSS,SPV,ANV,ANA,NOI
24072,I,2023-05-09 12:21:39.100,G,00:00:00.000,A,A4,900_228000498159,INL,NaN,0.8,703/703,1356.5,8,75,50
24073,S,2023-05-09 12:21:39.100,G,00:00:00.000,A,A4,900_228000498159,INL,NaN,0.8,703/703,1356.5,8,75,50
24175,S,2023-05-09 12:21:39.100,G,00:00:00.000,A,A4,900_228000498159,INL,NaN,0.8,703/703,1356.5,8,75,50
24074,I,2023-05-09 12:21:42.700,G,00:00:00.000,A,A4,900_228000498159,INL,NaN,0.8,698/698,1358.0,8,75,50
24075,S,2023-05-09 12:21:42.700,G,00:00:00.000,A,A4,900_228000498159,INL,NaN,0.8,698/698,1358.0,8,75,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24172,S,2023-05-09 12:28:11.900,G,00:00:00.000,A,A4,900_228000498176,INL,NaN,0.8,740/740,1358.5,8,75,50
24215,S,2023-05-09 12:28:11.900,G,00:00:00.000,A,A4,900_228000498176,INL,NaN,0.8,740/740,1358.5,8,75,50
24173,I,2023-05-09 12:28:13.900,G,00:00:00.000,A,A4,900_228000498176,INL,NaN,0.7,822/822,1358.0,7,75,50
24174,S,2023-05-09 12:28:13.900,G,00:00:00.000,A,A4,900_228000498176,INL,NaN,0.8,822/822,1358.0,7,75,50


Insights on duplicates from inspecting the raw files:
* Two tags have triplicate detections using a unique key of three columns: ['ARR', 'TAG', 'SCD']
* They come from files `INL 2023-05-09 12-05-47.txt` and `INL 2023-05-17 14-56-03.txt`
* In file `INL 2023-05-09 12-05-47.txt` they appear twice with differing `DTY` values.
* They appear once in `INL 2023-05-17 14-56-03.txt`
* We will drop duplicate values keeping the last.

In [6]:
# Drop duplicate values keeping the last
m_dups_drop = df_valid_fish.duplicated(subset=['ARR','TAG', 'SCD'], keep='last')
df_valid_fish = df_valid_fish[~m_dups_drop]
print(f"{m_dups_drop.sum()} duplicates dropped.")

103 duplicates dropped.


In [7]:
# Export Data
filename = 'combined_valid-tag_detections_2023data_2023-11-13.csv'
df_valid_fish.to_csv(processed_directory / filename, index=False)

In [8]:
df_valid_fish.TAG.value_counts()


TAG
900_228000487631    3503
900_228000487705    3488
900_228000487685    2999
900_228000487549    2795
900_228000487901    2309
                    ... 
900_228000487924       2
900_228000487927       1
900_228000498014       1
900_228000498180       1
900_228000498038       1
Name: count, Length: 213, dtype: int64

In [9]:
df_valid_fish.TAG.value_counts().reset_index().sort_values(by='TAG')

,TAG,count
14,900_228000487506,818
23,900_228000487515,498
3,900_228000487549,2795
88,900_228000487576,146
45,900_228000487581,266
...,...,...
20,900_228000498206,629
133,900_228000498207,56
27,900_228000498208,418
96,900_228000498210,134
